In [13]:
import os
import shutil
import numpy as np
import tensorflow as tf
from PIL import Image
from tqdm import tqdm

# --- CONFIGURACIÓN ---
# Edita estas rutas y nombres según tu proyecto

# Rutas de entrada (tu dataset en formato YOLO)
ORIGINAL_DATASET_DIR = os.path.join('yolo_dataset', 'val')

# Rutas del nuevo modelo clasificador
TM_LAND_MODEL_PATH = 'keras_model_land.h5'
TM_LAND_LABELS_PATH = 'labels_land.txt'

# Ruta de salida para el nuevo dataset filtrado
OUTPUT_DATASET_DIR = 'dataset_con_tierra'

# Parámetros de clasificación
CLASSIFIER_CONFIDENCE_THRESHOLD = 0.50  # Umbral para aceptar la clasificación "land"
POSITIVE_CLASS_NAME = "land"  # El nombre de tu clase positiva en labels_land.txt (ej. "Tierra", "Land", etc.)

# --- FUNCIONES DE AYUDA ---

def load_tm_model(model_path, labels_path):
    """Carga un modelo de Keras y sus etiquetas."""
    try:
        # Ocultar logs de TensorFlow para una salida más limpia
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
        model = tf.keras.models.load_model(model_path, compile=False)
        with open(labels_path, 'r') as f:
            # Asumimos que el formato es "0 Clase1", "1 Clase2", etc.
            labels = [line.strip().split(' ', 1)[1] for line in f]
        return model, labels
    except Exception as e:
        print(f"❌ Error al cargar el modelo de Teachable Machine desde '{model_path}': {e}")
        return None, None

def classify_image_with_tm(image_path, model, labels):
    """Preprocesa una imagen y la clasifica con el modelo de Teachable Machine."""
    try:
        image = Image.open(image_path).convert('RGB').resize((224, 224))
        image_array = np.asarray(image)
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        data[0] = normalized_image_array
        
        prediction = model.predict(data, verbose=0)
        index = np.argmax(prediction)
        class_name = labels[index]
        confidence_score = prediction[0][index]
        
        return class_name, confidence_score
    except Exception as e:
        print(f"Advertencia: No se pudo procesar la imagen {os.path.basename(image_path)}. Error: {e}")
        return None, 0

# --- FUNCIÓN PRINCIPAL DEL SCRIPT ---

def main():
    """
    Función principal que filtra un dataset en formato YOLO y crea uno nuevo.
    """
    print("--- Iniciando el script de filtrado de dataset YOLO por presencia de tierra ---")

    # 1. Cargar el modelo clasificador
    print(f"Cargando el modelo clasificador desde '{TM_LAND_MODEL_PATH}'...")
    land_classifier_model, land_classifier_labels = load_tm_model(TM_LAND_MODEL_PATH, TM_LAND_LABELS_PATH)
    if land_classifier_model is None:
        return

    # 2. Definir rutas de entrada y salida
    input_images_dir = os.path.join(ORIGINAL_DATASET_DIR, 'images')
    input_labels_dir = os.path.join(ORIGINAL_DATASET_DIR, 'labels')
    
    output_images_dir = os.path.join(OUTPUT_DATASET_DIR, 'images')
    output_labels_dir = os.path.join(OUTPUT_DATASET_DIR, 'labels')

    # 3. Preparar carpetas de salida
    print(f"Creando/limpiando la carpeta de salida del nuevo dataset: '{OUTPUT_DATASET_DIR}'")
    if os.path.exists(OUTPUT_DATASET_DIR):
        shutil.rmtree(OUTPUT_DATASET_DIR)
    os.makedirs(output_images_dir)
    os.makedirs(output_labels_dir)

    # 4. Filtrar imágenes y copiar archivos
    image_filenames = os.listdir(input_images_dir)
    images_kept_count = 0

    for image_filename in tqdm(image_filenames, desc="Clasificando y filtrando imágenes"):
        full_image_path = os.path.join(input_images_dir, image_filename)

        if not os.path.exists(full_image_path):
            continue

        # Clasificar la imagen completa
        class_name, confidence = classify_image_with_tm(full_image_path, land_classifier_model, land_classifier_labels)

        # --- LÍNEA CORREGIDA ---
        # Decidir si conservar la imagen usando una comparación exacta (==) en lugar de 'in'
        if class_name and class_name.lower() == POSITIVE_CLASS_NAME.lower() and confidence >= CLASSIFIER_CONFIDENCE_THRESHOLD:
            images_kept_count += 1
            
            # --- Copiar el archivo de imagen ---
            shutil.copy(full_image_path, os.path.join(output_images_dir, image_filename))
            
            # --- Copiar el archivo de etiqueta .txt correspondiente ---
            label_filename = os.path.splitext(image_filename)[0] + '.txt'
            full_label_path = os.path.join(input_labels_dir, label_filename)
            
            if os.path.exists(full_label_path):
                shutil.copy(full_label_path, os.path.join(output_labels_dir, label_filename))
            else:
                print(f"Advertencia: Se encontró la imagen '{image_filename}' pero no su etiqueta '{label_filename}'. No se copió la etiqueta.")

    print("\n¡Proceso completado con éxito!")
    print(f"Nuevo dataset creado en '{OUTPUT_DATASET_DIR}' con {images_kept_count} imágenes (y sus correspondientes etiquetas).")


# --- Punto de entrada del script ---
if __name__ == "__main__":
    main()

--- Iniciando el script de filtrado de dataset YOLO por presencia de tierra ---
Cargando el modelo clasificador desde 'keras_model_land.h5'...
Creando/limpiando la carpeta de salida del nuevo dataset: 'dataset_con_tierra'


Clasificando y filtrando imágenes: 100%|██████████| 1962/1962 [56:32<00:00,  1.73s/it]     


¡Proceso completado con éxito!
Nuevo dataset creado en 'dataset_con_tierra' con 562 imágenes (y sus correspondientes etiquetas).
